In [1]:
import pandas as pd

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#This code is for combining certain data from the necessary csv files into a single dataframe (complete)
pd.set_option('display.max_colwidth', None)

movies_full = pd.read_csv('newdata/movies_metadata.csv',usecols=("genres","id" ,"title","tagline", "overview","production_companies"),
                          dtype={"tagline": "string", "id":"string", 'genres':"string", "title": "string", "tagline": "string","overview":"string", "production_companies" :"string"})
ratings = pd.read_csv('newdata/ratings.csv', usecols = ("userId", "movieId", "rating"), dtype={"userId": "string","movieId": "string","rating": "string"})
ratings = ratings.rename(columns={"movieId": "id"})

keywords = pd.read_csv('newdata/keywords.csv', usecols = ("id", "keywords"), dtype={"id": "string","keywords":"string"})
credits = pd.read_csv("newdata/credits.csv", usecols = ("cast", "id"), dtype={"cast": "string", "id": "string"})

complete =  pd.merge(movies_full, ratings, on ="id")
complete =  pd.merge(complete,keywords, on ="id")
complete  = pd.merge(complete,credits, on ="id")


#new:
#userIds are in order
# every_id = list(unique(list(complete["userId"])))
#userIds are out of order
# sample_ids  = random.sample(every_id, 1000)
# completeNew = pd.DataFrame()
#This is a very expensive task...
#it is possile to choose a subset of users from here instead of
#ibcluing  the entire set of users
# for user in sample_ids:
#     completeNew = pd.concat([completeNew, complete.loc[complete["userId"] == user]])

#new:
# complete = complete.sample(frac=1, random_state = seed_int, axis =0)
#this is not the desired behavior
#the users ids need to show up in a true random order
# complete = complete.groupby(by = "userId", sort = False, group_keys = True).apply(lambda x: x)
#this is omitted since the values should not be sorted by userId just grouped by userId

complete = complete.sort_values(by = 'userId')

complete  = complete.dropna()

complete  = complete.loc[:,['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview" ]]


# print(complete.head())

# f = open("test_dicts.txt", "w", encoding="utf-8")
# f.write(str(list(complete["tagline"])))
# f.write(str(list(complete["overview"])))
# f.close()


In [2]:
import ast
import random

#seed for consistent results across runtime
# seed_int = 3
# random.seed(seed_int)

#used to filter out the rows of data with empty entries
def condition(array):
    length = len(array[4])
    if(array[4][length-2:] == "[]"):
        return False
    length = len(array[5])
    if(array[5][length-2:] == "[]"):
        return False
    length = len(array[6])
    if(array[6][length-2:] == "[]"):
        return False
    length = len(array[7])
    if(array[7][length-2:] == "[]"):
        return False   
    #this is probably not needed due to the dropNa function used above...
    # length = len(array[8])
    # if(array[8][length-4:]=="<NA>"):
    #     return False
    # length = len(array[9])
    # if(array[9][length-4:]=="<NA>"):
    #     return False 
    return True


#used to extract names
def populate_names(item):
    string  = item[1:-1]
    jsons = string.split("}, ")   
    names = ""
    cnt = 0
    for item in jsons:
        if(cnt == len(jsons)-1):
            temp_dict = ast.literal_eval(item)
            names+=str(temp_dict["name"])
        else:
            temp_dict = ast.literal_eval(item+"}")
            names+=str(str(temp_dict["name"])+" ")
        cnt += 1
    return names

#extract data from row of complete_array
def provide_data(array):
    movie_data = []
    movie_data.append(int(array[0]))
    movie_data.append(int(array[1]))
    movie_data.append(float(array[2]))
    movie_data.append(array[3])  

    movie_data.append(populate_names(array[4]))
    movie_data.append(populate_names(array[5]))
    movie_data.append(populate_names(array[6]))
    movie_data.append(populate_names(array[7]))

    movie_data.append(str(array[8]))
    movie_data.append(str(array[9]))
    return movie_data
    


#convert the dataframe into an array and build a dictionary
complete_array = complete.to_numpy()


gaps = []
size = 0
list_of_user_ids = []
last_id  = -1
past_first_it = False


#there seems to be a technical problem with gaps...
for row in complete_array:
    #need to omit the first iteration for gaps
    if(row[0]!= last_id):
        list_of_user_ids.append(row[0])
        last_id = row[0]
        if(past_first_it ==True):
            gaps.append(size)
            size =0 
    size+=1
    past_first_it = True

#there is always a gap for the last iteration
gaps.append(size)


index  = 0
user_to_data = dict()
#this is the total number of users in the whole dataset
total_nof_users = 261306
#this is the number of desired users before filtering
selected_nof_users_before_filter = 40000

avg =0
cnt =0


#instead of using this random pass generaion 
#the users can be picked randomly before hand and replace list_of_user_ids[i]
#will index still work???

#populate user_to_data from complete_array
for i in range(0, total_nof_users):
    #generate a random float to determine a pass for the user
    if (random.random()<float(selected_nof_users_before_filter/total_nof_users)):
        user_to_data[list_of_user_ids[i]] = []
        for j in range(index, len(complete_array)):
            if complete_array[j][0] == list_of_user_ids[i]:
                #condition is checked for complete_array[j]
                if(condition(complete_array[j])):
                    #this is where data is tranformed
                    transformed = provide_data(complete_array[j])
                    user_to_data[list_of_user_ids[i]].append(transformed)         
            else:
                avg += len(user_to_data[list_of_user_ids[i]])
                cnt+=1
                #this condition can be tweaked for better accuracy
                #len(user_to_data[list_of_user_ids[i]])<50 or len(user_to_data[list_of_user_ids[i]])>75
                if (len(user_to_data[list_of_user_ids[i]])<50 or len(user_to_data[list_of_user_ids[i]])>75):
                    del user_to_data[list_of_user_ids[i]]
                #note: changed from (index = j+1)
                index = j
                break
    else:
        #every iteration, index starts at first data point of the next user
        index += gaps[i]

#test for the last iteration not done in loop
if list_of_user_ids[total_nof_users-1] in user_to_data.keys():
    if (len(user_to_data[list_of_user_ids[i]])<50 or len(user_to_data[list_of_user_ids[i]])>75):
        del user_to_data[list_of_user_ids[total_nof_users-1]]


#needs to be sure that there are enough users after the condiiton
print(len(list(user_to_data.keys())))


#average number of ratings per user
print(float(avg/cnt))

#18 minutes
#2599 users



2599
29.675531515288107


In [4]:
#save in a file so that cells below can run without running this cell and above
import csv

with open("constructedData/constructedData.csv", "w", encoding="utf-8", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['userId','id','rating',"title", "genres","production_companies","keywords", "cast", "tagline", "overview"])
    for key in user_to_data.keys():
        writer.writerows(user_to_data[key])

In [24]:
#this is a starting point if the data is already saved to the constructedData.csv file
import csv

data_list =[]

with open("constructedData/constructedData.csv", 'r', encoding="utf-8") as read_obj:
    csv_reader = csv.reader(read_obj)
    data_list = list(csv_reader)

data_list = data_list[1:]


In [25]:
import random

#seed for consistent results across runtime
# seed_int = 1
# random.seed(seed_int)

#user to data rows 
user_to_data = dict()
user_to_data_train = dict()
user_to_data_test = dict()

user_id = -1
for row in data_list:
    if (row[0]!=user_id):
        user_id = row[0]
        user_to_data[row[0]] = [row]
    else:
        user_to_data[row[0]].append(row)

#this can be tweaked...
for i in range(1000):
    user = random.choice(list(user_to_data.keys()))
    user_to_data_train[user] = user_to_data[user]
    user_to_data.pop(user)


#for test data to be used later...
for i in range(200):
    user = random.choice(list(user_to_data_train.keys()))
    user_to_data_test[user] = user_to_data_train[user]
    user_to_data_train.pop(user)


user_to_data.clear()


In [26]:
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
import random
import json
from ordered_set import OrderedSet

#seed for consistent results across runtime
# seed_int = 1
# random.seed(seed_int)

user_to_movie_id_to_corpus_train = dict() #words in all movies for each user
user_to_movie_id_to_rating_train = dict() #ratings of all movies for each user
user_to_rand_movie_id_train  = dict() #movie id of the movie which rating is to be predicted by the model for each user

user_to_movie_id_to_corpus_test = dict() #words in all movies for each user
user_to_movie_id_to_rating_test = dict() #ratings of all movies for each user
user_to_rand_movie_id_test  = dict() #movie id of the movie which rating is to be predicted by the model for each user

movie_id_to_ratings_total = dict() #all ratings for each movie
movies_in_order = OrderedSet() #all movies in order
movie_id_to_average_rating_train = dict() #average movie rating amoung all users for each movie
user_to_average_rating = dict()
overall_sum_train = 0 #used to compute the overall_average
overall_counts_train = 0 #used to compute the overall_average


wnl = WordNetLemmatizer()


def populate_1(user_to_data, user_to_movie_id_to_corpus, user_to_movie_id_to_rating,
            movie_id_to_ratings, overall_sum, overall_counts, user_to_rand_movie_id):
    for user in user_to_data.keys():
        movie_strings_temp = dict()
        movie_id_to_rating_temp = dict()
        cnt = 0
        total =0
        rand_int = random.randint(0, len(user_to_data[user])-1)
        for movie_data in user_to_data[user]:
            if cnt == rand_int:    
                user_to_rand_movie_id[user] = movie_data[1]
            else:
                overall_sum += float(movie_data[2])
                overall_counts += 1
                total += float(movie_data[2])
            
            if movie_data[1] in movie_id_to_ratings.keys():
                movie_id_to_ratings[movie_data[1]].append(float(movie_data[2]))
            else:
                movie_id_to_ratings[movie_data[1]] = [float(movie_data[2])]

            movie_string = ""
            #avoid the first three data points (user id, movieid, and rating)
            #use only the text data
            for index in range (3,len(movie_data)):
                if(index!= len(movie_data)-1):
                    movie_string+= movie_data[index]+" "
                else:
                    movie_string+= movie_data[index]

            #need to try with and without lematization...
            cleaned = remove_stopwords(movie_string)
            cleaned = [wnl.lemmatize(word) for word in cleaned.split(" ")]
            cleaned = [word[:-1] for word in cleaned if word.endswith(".")] + [word for word in cleaned if not word.endswith(".")]
            
            movie_strings_temp[movie_data[1]] = cleaned.copy()
            movie_id_to_rating_temp[movie_data[1]] = float(movie_data[2])
            movies_in_order.add(movie_data[1])
            cnt+=1
        user_to_average_rating[user] = float(total/(cnt-1))
        user_to_movie_id_to_corpus[user] = movie_strings_temp.copy()
        user_to_movie_id_to_rating[user] = movie_id_to_rating_temp.copy()
    return overall_sum, overall_counts


overall_sum_train, overall_counts_train = populate_1(user_to_data_train, user_to_movie_id_to_corpus_train, 
                                                   user_to_movie_id_to_rating_train,movie_id_to_ratings_total,
                                                   overall_sum_train, overall_counts_train, user_to_rand_movie_id_train)

#this is used when there are no usable ratings for a movie
overall_average_train = float(overall_sum_train/overall_counts_train)

populate_1(user_to_data_test, user_to_movie_id_to_corpus_test, user_to_movie_id_to_rating_test,
            dict(), 0, 0, user_to_rand_movie_id_test)


#movie_id_to_average_rating_train is used by both the test and train set
for movie in movie_id_to_ratings_total.keys():
    temp = 0
    for rating in movie_id_to_ratings_total[movie]:
        temp +=rating
    movie_id_to_average_rating_train[movie] = float(temp/len(movie_id_to_ratings_total[movie]))


#used to see what the text data looks like...
file = open("test_dicts_1.txt", 'w', encoding="utf-8")
file.write(json.dumps(user_to_movie_id_to_corpus_train))
file.close()

file = open("test_dicts_2.txt", 'w', encoding="utf-8")
file.write(json.dumps(user_to_movie_id_to_corpus_test))
file.close()


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from ordered_set import OrderedSet
from collections import Counter


user_to_words_in_order_train = dict() #stores the words for each word that is part of the users rated movie corpus
user_to_word_counts_train = dict() #A list of words counts of the users "words_in_order" for each movie for a user
user_to_rand_word_counts_train = dict() #stores the word count of the movie with the rating to be predicted for each user

user_to_ratings_train = dict() #stores the ratings of the movies rated by the user, the rating to predict is marked with a -1
user_to_rating_to_predict_train = dict() #stores the actual rating of the movie to be predicted for each user
user_to_rand_index_train = dict() #stores the index of the movie with the rating to be predicted for each user

user_to_words_in_order_test = dict() #stores the words for each word that is part of the users rated movie corpus
user_to_word_counts_test = dict() #A list of words counts of the users "words_in_order" for each movie for a user
user_to_rand_word_counts_test = dict() #stores the word count of the movie with the rating to be predicted for each user

user_to_ratings_test = dict() #stores the ratings of the movies rated by the user, the rating to predict is marked with a -1
user_to_rating_to_predict_test = dict() #stores the actual rating of the movie to be predicted for each user
user_to_rand_index_test = dict() #stores the index of the movie with the rating to be predicted for each user


def populate_2(user_to_movie_id_to_corpus, user_to_rand_movie_id, user_to_movie_id_to_rating,
           user_to_rand_index, user_to_rating_to_predict, user_to_ratings):
    for user in user_to_movie_id_to_corpus.keys():
        temp = []
        cnt = 0
        for movie_id in user_to_movie_id_to_corpus[user]:
            if movie_id != user_to_rand_movie_id[user]:
                temp.append(user_to_movie_id_to_rating[user][movie_id])
            else:
                #this signifies the ratings to be predicted by the model
                user_to_rand_index[user] = cnt
                user_to_rating_to_predict[user] = user_to_movie_id_to_rating[user][movie_id]
                temp.append(-1)
            cnt+=1
        user_to_ratings[user] = temp.copy()


populate_2(user_to_movie_id_to_corpus_train, user_to_rand_movie_id_train, user_to_movie_id_to_rating_train,
           user_to_rand_index_train, user_to_rating_to_predict_train, user_to_ratings_train)
populate_2(user_to_movie_id_to_corpus_test, user_to_rand_movie_id_test, user_to_movie_id_to_rating_test,
           user_to_rand_index_test, user_to_rating_to_predict_test, user_to_ratings_test)


#need to think about this part...
#cossine similairty is relative to a user
#if two movies do not share a word then the cossine similarity is unchanged
#This means that extra words abscent in both movies do no change the outcome of the cossine similairty
#which means its ok to keep them in

def populate_3(user_to_movie_id_to_corpus, user_to_words_in_order):
    for user in user_to_movie_id_to_corpus.keys():
        user_to_words_in_order[user] = OrderedSet()
        for movie_id in user_to_movie_id_to_corpus[user].keys():
            for word in user_to_movie_id_to_corpus[user][movie_id]:
                user_to_words_in_order[user].add(word)

populate_3(user_to_movie_id_to_corpus_train,  user_to_words_in_order_train)
populate_3(user_to_movie_id_to_corpus_test,  user_to_words_in_order_test)


#note: user to word counts needs to omit movies with no rating

def populate_4(user_to_movie_id_to_corpus, user_to_word_counts, user_to_rand_movie_id, user_to_words_in_order, user_to_rand_word_counts):
    for user in user_to_movie_id_to_corpus.keys():
        user_to_word_counts[user] = []
        for movie_id in user_to_movie_id_to_corpus[user].keys():
            if movie_id != user_to_rand_movie_id[user]:
 
                #idea: instead of using words in order use a set of words that are local to the user
                #this cuts time iterating over words_in_order
                #and cuts space by having templist be shorter
                #note: it also adds a bit of space by needing a list of words for each user


                temp_dict = Counter(user_to_movie_id_to_corpus[user][movie_id])
                temp_list = []
                for word in user_to_words_in_order[user]:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                    else:
                        temp_list.append(0)               
                user_to_word_counts[user].append(temp_list)
            else:

                temp_dict = Counter(user_to_movie_id_to_corpus[user][movie_id])
                temp_list = []
                for word in user_to_words_in_order[user]:
                    if word in temp_dict.keys():
                        temp_list.append(temp_dict[word])
                    else:
                        temp_list.append(0)
                user_to_rand_word_counts[user] = temp_list.copy()

populate_4(user_to_movie_id_to_corpus_train, user_to_word_counts_train, user_to_rand_movie_id_train,
       user_to_words_in_order_train, user_to_rand_word_counts_train)

populate_4(user_to_movie_id_to_corpus_test, user_to_word_counts_test, 
       user_to_rand_movie_id_test, user_to_words_in_order_test, user_to_rand_word_counts_test)

#try removing omitting deletions...
del user_to_words_in_order_train
del user_to_words_in_order_test
del user_to_movie_id_to_corpus_train
del user_to_movie_id_to_corpus_test



#now for each user, use the user_to_index_full and find the most similair users by omitting that index across the
#sim_matrix



#collaboritive filtering idea
#data structures:
#need a user to movies to ratings dictionary (done) (only needed for train data)
#need a ordered set of all movies (done) (only needed for train data)

#need to transform it into a user to list of all movies with user ratings or otherwise filled in ratings with sutiable average
#(the list needs to be in a consistent order across users)
#to fill in the averages we need a movie to average rating dictionary
#(note: if no other user has rated the movie then fill it in with the overall movie average)
#standardize the data row wise (why not columnwise???)
#filter the users that have rated the movie to predict
#then use cossine similairity on this set to find the most similair user the the chosen user
#ignore the movie rating to predict with the cossine similarity function

#Note: this is an expensive task and the number of users may have to be truncated before running



#ideas: 
#idea1: 
#collaborative filtering:
#https://towardsdatascience.com/predict-movie-ratings-with-user-based-collaborative-filtering-392304b988af
#https://www.geeksforgeeks.org/user-based-collaborative-filtering/#
#https://www.youtube.com/watch?v=3ecNC-So0r4&ab_channel=CodeHeroku
#idea2: 
#note: the model can be scored based on how close a predition is to a threshold of .5 

#idea 3:
#there may be a replacement for cossine similarity:
#perhaps it is not properly matching similair people and similair movies 
#maybe there is a replacement function

#idea 4: 
#there should be more train and test users

#idea 5: (basic model)
#https://www.kaggle.com/code/muhammadayman/recommendation-system-using-cosine-similarity#Data-Cleaning



#10 minutes for 2500 users

In [64]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#train features
feature_1_train = []
feature_2_train = []
feature_3_train = []
true_rating_train = []

#test features
feature_1_test = []
feature_2_test = []
feature_3_test = []
true_rating_test = []

user_to_ratings_full_train = dict() #like "user_to_ratings_train" but includes movies the user did not rate with averages
user_to_rand_index_full_train  = dict() #like "user_to_rand_index_train" but with index that can span the entire set of movies

user_to_ratings_full_test = dict() #like "user_to_ratings_test" but includes movies the user did not rate with averages
user_to_rand_index_full_test  = dict() #like "user_to_rand_index_test" but with index that can span the entire set of movies


def predict(user, user_to_rand_word_counts, user_to_word_counts, user_to_ratings):

    cosine_sim = cosine_similarity(X = user_to_word_counts[user] ,Y = [user_to_rand_word_counts[user]])
    #not sure if reshape is needed???
    #should test with and without
    cosine_sim = np.reshape(cosine_sim,  (len(cosine_sim)))

    ratings = []

    for rating in user_to_ratings[user]:
        if rating != -1:
            ratings.append(rating)

    combined = zip(cosine_sim, ratings)
    combined = sorted(combined, key=lambda x: x[0], reverse=True)
    avg = 0
    nof = 10.0
    for i in range(int(nof)):
        avg += combined[i][1]

    return float(avg/nof)
    # for i in range(len(combined)):
    #     avg += combined[i][1]

    # return float(avg/len(combined))



def populate_5(user_to_movie_id_to_rating, feature_1,
                user_to_rand_word_counts, user_to_word_counts,
                user_to_ratings, true_rating, user_to_rating_to_predict):
    for user in user_to_movie_id_to_rating.keys():
        feature_1.append(predict(user, user_to_rand_word_counts, user_to_word_counts, user_to_ratings))
        true_rating.append(user_to_rating_to_predict[user])


populate_5(user_to_movie_id_to_rating_train, feature_1_train,
                user_to_rand_word_counts_train, user_to_word_counts_train,
                user_to_ratings_train, true_rating_train, user_to_rating_to_predict_train)

populate_5(user_to_movie_id_to_rating_test, feature_1_test,
                user_to_rand_word_counts_test, user_to_word_counts_test,
                user_to_ratings_test, true_rating_test, user_to_rating_to_predict_test)


#populates feature_2_train
for user in user_to_movie_id_to_rating_train.keys(): 
    if(len(movie_id_to_ratings_total[user_to_rand_movie_id_train[user]])==1):
        #if there are no other ratings besides the user in questions rating
        feature_2_train.append(overall_average_train)
    else:
        #omit the rating the user in question made
        feature_2_train.append(float(((movie_id_to_average_rating_train[user_to_rand_movie_id_train[user]]
                        *len(movie_id_to_ratings_total[user_to_rand_movie_id_train[user]]))
                        -user_to_movie_id_to_rating_train[user][user_to_rand_movie_id_train[user]])
                        /(len(movie_id_to_ratings_total[user_to_rand_movie_id_train[user]])-1)))

#populates feature_2_test
for user in user_to_movie_id_to_rating_test.keys():
    if(user_to_rand_movie_id_test[user] not in movie_id_to_ratings_total.keys()):
        #if the movie is not in the train set make a guess
        feature_2_test.append(overall_average_train)
    else:
        #only use the train data
        feature_2_test.append(movie_id_to_average_rating_train[user_to_rand_movie_id_test[user]])



def populate_6(user_to_ratings_full, user_to_rand_index_full, user_to_movie_id_to_rating, user_to_rand_movie_id):
    for user in user_to_movie_id_to_rating.keys():
        ratings = []
        index = 0
        for movie_id in movies_in_order:
            #note: when this happens should rating be omitted???
            #it is omitted in another way later
            #why can it not be omitted now???
            if movie_id == user_to_rand_movie_id[user]:
                user_to_rand_index_full[user] = index
            if movie_id in user_to_movie_id_to_rating[user].keys():
                ratings.append(user_to_movie_id_to_rating[user][movie_id])
            elif movie_id in movie_id_to_average_rating_train.keys():
                #note: does not include test ratings
                ratings.append(movie_id_to_average_rating_train[movie_id])
            else:   
                #note: does not include test ratings
                ratings.append(overall_average_train)
            index +=1
        user_to_ratings_full[user] = ratings.copy()


populate_6(user_to_ratings_full_train, user_to_rand_index_full_train, user_to_movie_id_to_rating_train, user_to_rand_movie_id_train)
populate_6(user_to_ratings_full_test, user_to_rand_index_full_test, user_to_movie_id_to_rating_test, user_to_rand_movie_id_test)


def populate_7(user_to_ratings_full, user_to_rand_index_full, feature_3):
    for user_1 in user_to_ratings_full.keys():
        list_of_ratings = []
        hidden_ratings = []
        for user_2 in user_to_ratings_full_train.keys():
            if user_1 != user_2:
                ratings = user_to_ratings_full_train[user_2].copy()
                hidden_ratings.append(ratings[user_to_rand_index_full[user_1]])
                ratings = [x - user_to_average_rating[user_2] for x in ratings]
                del ratings[user_to_rand_index_full[user_1]]
                list_of_ratings.append(ratings.copy())
        user_ratings = user_to_ratings_full[user_1].copy()
        user_ratings = [x - user_to_average_rating[user_1] for x in user_ratings]
        del user_ratings[user_to_rand_index_full[user_1]]
        sim  = cosine_similarity(X = list_of_ratings ,Y = [user_ratings])
        sim = np.reshape(sim,  (len(sim)))

        combined = zip(sim, hidden_ratings)
        combined = sorted(combined, key=lambda x: x[0], reverse=True)

        avg = 0
        nof = 10.0
        for i in range(int(nof)):
            avg+= combined[i][1]

        feature_3.append(float(avg/nof))
    

#populate feature_3_train and feature_3_test
populate_7(user_to_ratings_full_train, user_to_rand_index_full_train, feature_3_train)
populate_7(user_to_ratings_full_test, user_to_rand_index_full_test, feature_3_test)

#adjusted cossine simlairity
#https://stackoverflow.com/questions/40716459/choice-between-an-adjusted-cosine-similarity-vs-regular-cosine-similarity
#https://github.com/csaluja/JupyterNotebooks-Medium/blob/master/CF%20Recommendation%20System-Examples.ipynb?source=post_page-----ecbffe1c20b1--------------------------------
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1


#note: in the item-based collabortive filtering the means is implementaed item-wise
#this means with user-based collaboritve filtering the mean is implementad user-wise

#there are other options availble


In [83]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score


#always the same 
# scalar = StandardScaler()

#need more
# hyperparams = [[(10,10,10), (20,15,10)], ["scale", "no_scale"]]

#feature scaling:
#https://analyticsindiamag.com/why-data-scaling-is-important-in-machine-learning-how-to-effectively-do-it/#:~:text=Scaling%20the%20target%20value%20is,learn%20and%20understand%20the%20problem.&text=Scaling%20of%20the%20data%20comes,algorithms%20in%20the%20data%20set.
#https://towardsdatascience.com/collaborative-filtering-based-recommendation-systems-exemplified-ecbffe1c20b1


#without scaling...
# reg = MLPRegressor(hidden_layer_sizes = (20,15,10), max_iter = 10000)
# train_inputs = []
# for feature_1, feature_2 in zip(feature_1_train, feature_3_train):
#     train_inputs.append([feature_1, feature_2])
# reg.fit(train_inputs, true_rating_train)
# results = permutation_importance(reg, train_inputs,true_rating_train)
# print(results["importances_mean"])
# test_inputs = []
# for feature_1, feature_2 in zip(feature_1_test, feature_3_test):
#     test_inputs.append([feature_1, feature_2])
# predictions = reg.predict(test_inputs)
# rounded_predictions = []
# for item in predictions:
#     rounded_predictions.append(float(round(item*2)/2.0))
# print(r2_score(true_rating_test, predictions))
# print(r2_score(true_rating_test, rounded_predictions))

train_inputs = []
for feature_1, feature_2 in zip(feature_1_train, feature_3_train):
    train_inputs.append([feature_1, feature_2])

input_scalar = StandardScaler()
train_inputs = input_scalar.fit_transform(train_inputs)

target_scalar = StandardScaler()
true_rating_train_scaled = target_scalar.fit_transform(np.array(true_rating_train).reshape(-1, 1))

true_rating_train_scaled = np.reshape(true_rating_train_scaled, len(true_rating_train_scaled))

reg = MLPRegressor(hidden_layer_sizes = (10,10,10), max_iter = 10000)


reg.fit(train_inputs, true_rating_train_scaled)

results = permutation_importance(reg, train_inputs,true_rating_train_scaled)

print(results["importances_mean"])

test_inputs = []
for feature_1, feature_2 in zip(feature_1_test, feature_3_test):
    test_inputs.append([feature_1, feature_2])

input_scalar = StandardScaler()
test_inputs = input_scalar.fit_transform(test_inputs)

scaled_predictions = reg.predict(test_inputs)

predictions = target_scalar.inverse_transform(np.array(scaled_predictions).reshape(-1, 1))

rounded_predictions = []
for item in predictions:
    rounded_predictions.append(float(round(item[0]*2)/2.0))

print(r2_score(true_rating_test, predictions))
print(r2_score(true_rating_test, rounded_predictions))


[0.09319426 0.33755663]
0.2728846770073057
0.22376319602566752


In [66]:
#this cell is for observations and process descriptions


#How will the model work to predict ratings in the test data???
#for each test user a movie is randomly selected

#The prediction will be made with the word counts for the ratings by the user but using the
#the words_in_order of the train data for faster runtime (may change later)

#besides an average for a number of simailair movies the user has watched, the easy
#parameter is average rating since it only considers ratings from train data





# commented out for testing
# predicted = []
# for user in user_to_movie_id_to_corpus_train.keys(): 
#     if(len(movie_id_to_ratings[user_to_rand_movie_id[user]])==1):
#         predicted.append(overall_average)
#     else:
#         predicted.append(float(((movie_id_to_average_rating[user_to_rand_movie_id[user]]
#                         *len(movie_id_to_ratings[user_to_rand_movie_id[user]]))
#                         -user_to_movie_id_to_rating[user][user_to_rand_movie_id[user]])
#                         /(len(movie_id_to_ratings[user_to_rand_movie_id[user]])-1)))
        
# nof_overall_avg = 0
# for item in predicted:
#     if item == overall_average:
#         nof_overall_avg += 1
# print(nof_overall_avg)
# print(r2_score(true, predicted))


#now for the user comparison logic (need user to list of movie ratings)
#fill in ratings that the user hasn't watched with the method above
#then cluster the users by their ratings

#note: agglomerative clustering might make more sense here since k-means has random init for centroids...
#note: to guess a new users rating requires that none of that users ratings have been used to train the model
#The data needs to be split into test and train before modeling the algorithm on the train data

#Training process:
#split data into test and train data
#proceed with train data...
#cluster movies by the tokens with range for k
#cluster users by the ratings with range for k and (fill in ratings for movies a users hasn't watched with some guess)
#guess: this can be obtained by clustering the movies that the user has watched...
#for each movie the user hasn't watched find the cluster that it belongs to with the highest possible k value
#that the user has at least one movie belonging to one of the clusters and then take the average of those movies
#this is exactly like a later training step excpet it is applied to all the movies the user watched

#for a single randomly chosen movie from each user in the trainging data...

#find the cluster the movie belongs to 
#find the movies part of that same cluster that the user has scored at the highest possible k value
#take the average score of these movies
#find the cluster the user belongs to
#find the average rating of the movie for users in that cluster at the highest possible k value
#train an mlp model with both averages and perhaps some extra statistics as features...
#using the given movie ratings as actuals


#The process of predicting a rating:
#1. find the cluster the movie belongs to 
#2. find the movies part of that same cluster that the user has scored at the highest possible k value
#3. take the average score of these movies
#4. find the cluster the user belongs to
#5. find the average rating of the movie for users in that cluster at the highest possible k value
#6. input into the trained mlp model both averages and perhaps some extra statistics
#7. make predictions and test against the randomly chosen movies actual ratings


#summary:
#find cluster for movie -> find movies part of the same clusters that the users rated -> average
#question: are the clusters unique to the movies the user has watched or to all movies???
#what is the technical difference???
#is this the same as finding the most simimlair movie the user rated and copying the rating???

#find cluster for user -> find the ratings for the movie by people in the same cluster -> average

#other avenues considered:
#idea 1:
#for the first process, instead of averaging the movies that only the user rated, find other users that are...
#like the user in question and find the average for that movie cluster
#Problem: it is better to get the users raw opionion rather than generalizing it to some like minded users
#there is an extra costly step to this
#idea 2: 
#for the second process, instead of finding the average rating for the movie in the same cluster of users...
#also find the average rating of movies that are like the movie in question 
#Problem, it is better to get the movies rating itself as it would be the most accurate indicator
#there is an extra costly step to this




#test with overall averges:
#note: users have between 50 and 75 ratings each
#note: there are 250 users who are taken into account 
#note: seed int is one for above cells (cells part of creating the csv file)

#with seed int == 3, 4 taking overall averages: 0.09565753948597455
#with seed_int == 1, 2 taking overall averages: 0.070404868516315
#with seed_int == 2, 5 taking overall averages: 0.11310085954932936
#with seed_int == 4, 6 taking overall averages: 0.07125374341347135
#with seed_int == 5, 4 taking overall averages: 0.17736444913943628
#compute time: 11 minutes


#test with users related movies:
#is there a magic proportion of movies to average???
#note: this is taking around the same time as the above tests meaning 
#there could be more users to include in analysis with little increase in runtime
#k fold cross validation could be effective
#https://www.youtube.com/watch?v=TIgfjmp-4BA&ab_channel=Udacity

#effect of choosing a random seed...
#https://towardsdatascience.com/how-to-use-random-seeds-effectively-54a4cd855a79

#try tinkering with the number of similair movies to average
